In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install pydot graphviz

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os


BASE_DIR = "/content/drive/MyDrive/Project/Image Captioning" 

WORKING_DIR = os.path.join(BASE_DIR, "Inception-v3")

os.chdir(WORKING_DIR)



In [ ]:
WORKING_DIR

In [ ]:
import os


BASE_DIR = "/kaggle/input/flickr8k"

WORKING_DIR = os.getcwd()

print("Working Directory:", WORKING_DIR)
print("Base Directory:", BASE_DIR)

In [ ]:
pip install tensorflow==1.14

In [ ]:


tf.executing_eagerly()


In [ ]:
import pickle
import numpy as np
import tensorflow as tf
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Embedding, LSTM, Dropout, Flatten, Input, add
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical, plot_model

In [ ]:
import pandas as pd

captions_df = pd.read_csv(WORKING_DIR + "/Data/captions.txt")

captions_df.head(10)

In [ ]:
import re
for i in captions_df["image"]:
    x = re.search("([^\s]+(\.(?i)(jpg|png|gif|bmp))$)", i)
    if (x):
        pass
    else:
        print(f"YES! We have a match!: {i}")

In [ ]:
captions_df.caption.values

In [ ]:
# Importing InceptionV3 model with pre-trained ImageNet weights
inception_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

# Getting the input and output layers of the InceptionV3 model
new_input = inception_model.input
hidden_layer = inception_model.layers[-1].output

# Creating a feature extraction model using the input and output layers
image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

# Displaying the summary of the feature extraction model
image_features_extract_model.summary()


In [ ]:
os.makedirs('/kaggle/working/flickr8k')

In [ ]:
import shutil
shutil.rmtree('/kaggle/working/flickr8k')

In [ ]:
import shutil

# Specify the source folder path from the dataset
src_folder = '/kaggle/input/'

# Specify the destination folder path in your working directory
dst_folder = WORKING_DIR + "/Data"

# Copy the folder and its contents to the working directory
shutil.copytree(src_folder, dst_folder)

In [ ]:
BASE_DIR = '/kaggle/working/Data'

In [ ]:
dir = BASE_DIR + '/Images'

img_paths = []

for img_name in os.listdir(dir):
    img_path = dir + '/' + img_name
    img_paths.append(img_path)

print(len(img_paths))
print(img_paths[0])

In [ ]:
#from keras.applications.vgg16 import preprocess_input

In [ ]:
from tensorflow.keras.applications.inception_v3 import preprocess_input

In [ ]:
def load_image(image_path):
    image_path_str = tf.strings.as_string(image_path)  # Convert tensor to string tensor
    image_name = tf.strings.split(tf.strings.split(image_path_str, '/')[-1], '.')[0]  # Extract image name
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = preprocess_input(img)
    return img, image_name

img = load_image(BASE_DIR + '/Images/3582465732_78f77f34ae.jpg')[0]
print(img.shape)


In [ ]:
image_dataset = tf.data.Dataset.from_tensor_slices(img_paths)
image_dataset = image_dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(64)

print(f"Let's check the BatchDataset shapes--> {image_dataset}")


In [ ]:
image_dataset

In [ ]:
# %%time
# for img, path in tqdm(image_dataset):
#   batch_features = image_features_extract_model(img)
#   batch_features = tf.reshape(batch_features,
#                               (batch_features.shape[0], -1, batch_features.shape[3]))

#   for bf, p in zip(batch_features, path):
#     path_of_feature = p.numpy().decode("utf-8")
#     np.save(path_of_feature, bf.numpy())

In [ ]:
import shutil

save_dir = 'kaggle/working/Features'

#shutil.rmtree('/kaggle/working/flickr8k')

In [ ]:
 os.makedirs('Inception_Features', exist_ok = True)

In [ ]:
os.remove('/kaggle/working/Features/Tensor("args_0:0", shape=(), dtype=string).pkl')

In [ ]:
import shutil

shutil.rmtree('/content/drive/MyDrive/Project/Image Captioning/checkpoint_final')

In [ ]:
# for item in image_dataset.take(1):
#     print(item)

In [ ]:
import pickle

save_dir = tf.constant(WORKING_DIR + '/Inception_Features')
save_dir = save_dir.numpy().decode('utf-8')

for img, name in tqdm(image_dataset):
  batch_features = image_features_extract_model(img)
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

  for bf, image_name in zip(batch_features, name):
    image_feature_name = image_name.numpy().decode('utf-8')
    #image_feature_name = p.numpy().decode('utf-8')
    #full_path = os.path.join(save_dir, image_feature_name)
    #print(save_dir)
    #print(image_feature_name)

    with open(os.path.join(save_dir, image_feature_name + '.pkl'),'wb') as f:
        pickle.dump(bf,f)


In [ ]:
count = 0

save_dir = WORKING_DIR + '/Inception_Features'

for i in os.listdir(save_dir):
    count += 1

print(count)


In [ ]:
tf.data.Dataset.from_tensor_slices?

In [ ]:
bf.numpy().shape

In [ ]:
batch_features.numpy().shape

In [ ]:
len(captions_df.caption)

In [ ]:
len(set(captions_df.image))

In [ ]:
def get_name(path):
    name = os.path.basename(path)
    name = name.split('.')[0]
    return name

name = get_name(BASE_DIR + '/Images/2567035103_3511020c8f.jpg')
print(name)

In [ ]:
all_captions = []

for caption in captions_df['caption']:
    caption = "<sos> " + caption + " <eos>"
    all_captions.append(caption)

all_captions[:10]

In [ ]:
image_names = []

for name in captions_df['image']:
    image_name = get_name(name)
    image_names.append(image_name)

image_names[:10]

In [ ]:
from sklearn.utils import shuffle

shuffled_captions, shuffled_image_names = shuffle(all_captions,
                                                  image_names,
                                                  random_state=1)

In [ ]:
#len(shuffled_captions)


In [ ]:
# len(shuffled_image_names)
# shuffled_image_names[:10]

In [ ]:
#function for getting maxlen of the caption

def get_maxlen(captions):
    len_caption = []
    for caption in captions_df['caption']:
        len_caption.append(len(caption.split()))

    return max(len_caption)

In [ ]:
# utilizing the tokenizer

top_k = 5000
tokenizer = Tokenizer(num_words = top_k, oov_token = '<unk>', filters = '!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')

tokenizer.fit_on_texts(shuffled_captions)

tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

train_seqs = tokenizer.texts_to_sequences(shuffled_captions)

vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)


In [ ]:
tokenizer.word_index['<pad>']

In [ ]:
tokenizer.index_word

In [ ]:
#tokenizer.word_index

In [ ]:
max_length = get_maxlen(train_seqs)
print(max_length)

In [ ]:
caption_vector = pad_sequences(train_seqs, maxlen = max_length, padding = 'post')
print(caption_vector.shape)

In [ ]:
# train test split

from sklearn.model_selection import train_test_split

train_image, test_image, train_caption, test_caption = train_test_split(shuffled_image_names,
                                                                        caption_vector,
                                                                       test_size = 0.2,
                                                                       random_state = 0)

In [ ]:
len(train_image), len(test_image), len(train_caption), len(test_caption)

In [ ]:
train_image[0]

In [ ]:
train_caption[0]

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 1000
embedding_dim = 256
units = 512
num_steps = len(train_image) // BATCH_SIZE
features_shape = 2048
attention_features_shape = 64

In [ ]:

save_dir = tf.constant(WORKING_DIR + '/Inception_Features')
save_dir = save_dir.numpy().decode('utf-8')

def map_func(image_name, caption):
    name = tf.constant(image_name)
    image_name_str = name.numpy().decode('utf-8')
    with open(os.path.join(str(save_dir), image_name_str + ".pkl"),'rb') as f:
        img_tensor = pickle.load(f)
    return img_tensor, caption

tensor, cap = map_func(train_image[0], train_caption[0])
print(tensor)
print(cap)


In [ ]:
# for i in os.listdir(features_dir):
#     name = i.split('.')[0]
#     if name == train_image[0]:
#         print("found")
#         break
#     else:
#         continue



In [ ]:

dataset = tf.data.Dataset.from_tensor_slices((train_image, train_caption))

# Use map to load the numpy files in parallel
dataset = dataset.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Shuffle and batch
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
for item in dataset.take(1):
    print(item)

In [ ]:
len(dataset)

In [ ]:
tf.data.experimental.enable_debug_mode()

# Model


In [ ]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)

    # hidden shape == (batch_size, hidden_size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(hidden, 1)

    # score shape == (batch_size, 64, hidden_size)
    score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

    # attention_weights shape == (batch_size, 64, 1)
    # you get 1 at the last axis because you are applying score to self.V
    attention_weights = tf.nn.softmax(self.V(score), axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

class CNN_Encoder(tf.keras.Model):
    # Since you have already extracted the features and dumped it using pickle
    # This encoder passes those features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x

class RNN_Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.lstm = tf.keras.layers.LSTM(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.fc2 = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.units)

  def call(self, x, features, hidden):
    # defining attention as a separate model
    context_vector, attention_weights = self.attention(features, hidden)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state, c_state = self.lstm(x)

    # shape == (batch_size, max_length, hidden_size)
    x = self.fc1(output)

    # x shape == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))

    # output shape == (batch_size * max_length, vocab)
    x = self.fc2(x)

    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))

In [ ]:
# BATCH_SIZE = 64
# BUFFER_SIZE = 1000
# embedding_dim = 256
# units = 512
# num_steps = len(train_image) // BATCH_SIZE
# features_shape = 2048
# attention_features_shape = 64

encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

In [ ]:
encoder.summary()

In [ ]:
import tensorflow as tf

# Plot the CNN Encoder model
tf.keras.utils.plot_model(encoder, to_file='encoder_model.png', show_shapes=True)


In [ ]:
# Plot the RNN Decoder model

tf.keras.utils.plot_model(decoder, to_file='decoder_model.png', show_shapes=True)


In [ ]:
decoder.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
# saving checkpoint

checkpoint_path_ckpt = WORKING_DIR + "/checkpoint_final/train"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path_ckpt, max_to_keep=5)

In [ ]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
  start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])

In [ ]:
print(start_epoch)

# Training

In [ ]:
# Function to save the entire model
def save_model(encoder, decoder):
    tf.saved_model.save(encoder, ckpt_manager.save_checkpoint_path)
    tf.saved_model.save(encoder, ckpt_manager.save_checkpoint_path)


In [ ]:
import time

loss_plot = []

@tf.function
def train_step(img_tensor, target):
    loss = 0

  # initializing the hidden state for each batch
  # because the captions are not related from image to image
    hidden = decoder.reset_state(batch_size=target.shape[0])

    dec_input = tf.expand_dims([tokenizer.word_index['<sos>']] * target.shape[0], 1)

    with tf.GradientTape() as tape:
        features = encoder(img_tensor)

        for i in range(1, target.shape[1]):
            # passing the features through the decoder
            predictions, hidden, _ = decoder(dec_input, features, hidden)

            loss += loss_function(target[:, i], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(target[:, i], 1)

        total_loss = (loss / int(target.shape[1]))

        trainable_variables = encoder.trainable_variables + decoder.trainable_variables

        gradients = tape.gradient(loss, trainable_variables)

        optimizer.apply_gradients(zip(gradients, trainable_variables))

        return loss, total_loss


EPOCHS = 20

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(
              epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    if epoch % 5 == 0:
        ckpt_manager.save()
        

    print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                         total_loss/num_steps))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))


In [ ]:
start_epoch = 21

In [ ]:
import time

loss_plot = []

@tf.function
def train_step(img_tensor, target):
    loss = 0

  # initializing the hidden state for each batch
  # because the captions are not related from image to image
    hidden = decoder.reset_state(batch_size=target.shape[0])

    dec_input = tf.expand_dims([tokenizer.word_index['<sos>']] * target.shape[0], 1)

    with tf.GradientTape() as tape:
        features = encoder(img_tensor)

        for i in range(1, target.shape[1]):
            # passing the features through the decoder
            predictions, hidden, _ = decoder(dec_input, features, hidden)

            loss += loss_function(target[:, i], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(target[:, i], 1)

        total_loss = (loss / int(target.shape[1]))

        trainable_variables = encoder.trainable_variables + decoder.trainable_variables

        gradients = tape.gradient(loss, trainable_variables)

        optimizer.apply_gradients(zip(gradients, trainable_variables))

        return loss, total_loss


EPOCHS = 40

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(
              epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    if epoch % 5 == 0:
        ckpt_manager.save()
        

    print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                         total_loss/num_steps))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))


In [ ]:
ckpt_manager.save()

In [ ]:
with open(WORKING_DIR + '/loss_plot.pkl','wb') as f:
    pickle.dump(loss_plot, f)

In [ ]:
checkpoint_path_ckpt = WORKING_DIR + "/checkpoint_final/train"

checkpoint_path = checkpoint_path_ckpt
#tf.compat.v2.keras.callbacks.ModelCheckpoint
ckpt = tf.train.Checkpoint(encoder=encoder,
                               decoder=decoder,
                               optimizer = optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path_ckpt, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    # Restore the latest checkpoint
    ckpt.restore(ckpt_manager.latest_checkpoint)


In [ ]:
with open(WORKING_DIR + '/loss_plot.pkl', 'rb') as f:
    loss_plot = pickle.load(f)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Plot')
plt.show()

In [ ]:
import json
from glob import glob
from PIL import Image

def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

#     img_path = tf.constant(BASE_DIR + '/Images/' + image)
#     print(img_path)
#     img = load_image(img_path)[0]

#     temp_input = tf.expand_dims(img, 0)
#     img_tensor_val = image_features_extract_model(temp_input)

    name = image.split('.')[0]

    with open(WORKING_DIR + '/Inception_Features/' + name + ".pkl","rb") as t:
        img_tensor_val = pickle.load(t)
        
    #img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<sos>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<eos>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

In [ ]:
def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(os.path.join(BASE_DIR, 'Images',image)))

    fig = plt.figure(figsize=(10, 10))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (8, 8))
        ax = fig.add_subplot(len_result//2, len_result//2, l+1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    #plt.tight_layout()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def predict_caption(image):
  real_caption = ' '.join([tokenizer.index_word[i] for i in test_caption[rid] if i not in [0]])
  result, attention_plot = evaluate(image)

  for i in result:
      if i=="<unk>":
          result.remove(i)

  print ('Real Caption:', real_caption)
  print ('Prediction Caption:', ' '.join(result))
  plot_attention(image, result, attention_plot)
  # opening the image
  #Image.open(os.path.join(WORKING_DIR, 'Images', image + ".jpg"))
  path = os.path.join(BASE_DIR, 'Images', image)
  img = mpimg.imread(path)
  plt.imshow(img)


In [ ]:
rid = np.random.randint(0, len(test_image))
image = test_image[rid] + '.jpg'

predict_caption(image)

In [ ]:
rid = np.random.randint(0, len(test_image))
image = test_image[rid] + '.jpg'

predict_caption(image)

In [ ]:
rid = np.random.randint(0, len(test_image))
image = test_image[rid] + '.jpg'

predict_caption(image)

In [ ]:
rid = np.random.randint(0, len(test_image))
image = test_image[rid] + '.jpg'

predict_caption(image)

In [ ]:
rid = np.random.randint(0, len(test_image))
image = test_image[rid] + '.jpg'

predict_caption(image)


In [ ]:
rid = np.random.randint(0, len(test_image))
image = test_image[rid] + '.jpg'

predict_caption(image)

In [ ]:
rid = np.random.randint(0, len(test_image))
image = test_image[rid] + '.jpg'

predict_caption(image)

In [ ]:
rid = np.random.randint(0, len(test_image))
image = test_image[rid] + '.jpg'

predict_caption(image)

In [ ]:
rid = np.random.randint(0, len(test_image))
image = test_image[rid] + '.jpg'

predict_caption(image)

In [ ]:
rid = np.random.randint(0, len(test_image))
image = test_image[rid] + '.jpg'

predict_caption(image)

In [ ]:
rid = np.random.randint(0, len(test_image))
image = test_image[rid] + '.jpg'

predict_caption(image)

In [ ]:
rid = np.random.randint(0, len(test_image))
image = test_image[rid] + '.jpg'

predict_caption(image)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
reference = [['this', 'a', 'test','of', 'cricket', 'match']]
candidate = ['this', 'is', 'a', 'test','of', 'cricket', 'match']
print('Individual 1-gram: %f' % sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)))
print('Individual 2-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 1, 0, 0)))
print('Individual 3-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 1, 0)))
print('Individual 4-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 0, 1)))

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
import time

rid = np.random.randint(0, len(test_image))
image = test_image[rid] + '.jpg'

start = time.time()
real_caption = ' '.join([tokenizer.index_word[i] for i in test_caption[rid] if i not in [0]])
result, attention_plot = evaluate(image)


first = real_caption.split(' ', 1)[1]
real_caption = first.rsplit(' ', 1)[0]

#remove "<unk>" in result
for i in result:
    if i=="<unk>":
        result.remove(i)


#remove <end> from result
result_join = ' '.join(result)
result_final = result_join.rsplit(' ', 1)[0]

real_appn = []
real_appn.append(real_caption.split())
reference = real_appn
candidate = result_final
score = sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0))
print(f"BLEU score: {score}")

print ('Real Caption:', real_caption)
print ('Prediction Caption:', result_final)
plot_attention(image, result, attention_plot)

print(f"time took to Predict: {round(time.time()-start)} sec")
# opening the image
path = os.path.join(BASE_DIR, 'Images', image)
img = mpimg.imread(path)
plt.imshow(img)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
import time

rid = np.random.randint(0, len(test_image))
image = test_image[rid] + '.jpg'

start = time.time()
real_caption = ' '.join([tokenizer.index_word[i] for i in test_caption[rid] if i not in [0]])
result, attention_plot = evaluate(image)


first = real_caption.split(' ', 1)[1]
real_caption = first.rsplit(' ', 1)[0]

#remove "<unk>" in result
for i in result:
    if i=="<unk>":
        result.remove(i)


#remove <end> from result
result_join = ' '.join(result)
result_final = result_join.rsplit(' ', 1)[0]

real_appn = []
real_appn.append(real_caption.split())
reference = real_appn
candidate = result_final
score = sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0))
print(f"BLEU score: {score*100}")

print ('Real Caption:', real_caption)
print ('Prediction Caption:', result_final)
plot_attention(image, result, attention_plot)

print(f"time took to Predict: {round(time.time()-start)} sec")
# opening the image
path = os.path.join(BASE_DIR, 'Images', image)
img = mpimg.imread(path)
plt.imshow(img)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
import time

rid = np.random.randint(0, len(test_image))
image = test_image[rid] + '.jpg'

start = time.time()
real_caption = ' '.join([tokenizer.index_word[i] for i in test_caption[rid] if i not in [0]])
result, attention_plot = evaluate(image)


first = real_caption.split(' ', 1)[1]
real_caption = first.rsplit(' ', 1)[0]

#remove "<unk>" in result
for i in result:
    if i=="<unk>":
        result.remove(i)


#remove <end> from result
result_join = ' '.join(result)
result_final = result_join.rsplit(' ', 1)[0]

real_appn = []
real_appn.append(real_caption.split())
reference = real_appn
candidate = result_final
score = sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0))
print(f"BLEU score: {score*100}")

print ('Real Caption:', real_caption)
print ('Prediction Caption:', result_final)
plot_attention(image, result, attention_plot)

print(f"time took to Predict: {round(time.time()-start)} sec")
# opening the image
path = os.path.join(BASE_DIR, 'Images', image)
img = mpimg.imread(path)
plt.imshow(img)

In [ ]:
rid = np.random.randint(0, len(test_image))
image = test_image[rid] + '.jpg'

predict_caption(image)

In [ ]:
from IPython.display import FileLink
FileLink(r'file.zip')

In [ ]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.path.join(WORKING_DIR), file_name = 'Features.zip'):
    """
    zip all the files in a directory

    Parameters
    _____
    directory: str
        directory needs to be zipped, defualt is current working directory

    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'

    Returns
    _____
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)


In [ ]:
#!zip -r file.zip /kaggle/working/Features

In [ ]:
# reading captions
import os 

with open(os.path.join(BASE_DIR,"captions.txt"),"r") as c:
  next(c)
  read_captions = c.read() # reading the file 'captions.txt' into read_captions

In [ ]:
# creating mapping of image to their captions

from tqdm.notebook import tqdm

def create_mapping(doc):
  mapping = {}

  for line in tqdm(doc.split("\n")):
    if len(line) < 2:
      continue
    words = line.split(",")
    img_name = words[0]
    image_id = img_name.split(".")[0]

    caption = words[1:]
    caption = " ".join(caption)

    if image_id not in mapping:
      mapping[image_id] = []

    mapping[image_id].append(caption)

  return mapping

mapping = create_mapping(read_captions)


In [ ]:
import pickle

with open(os.path.join(WORKING_DIR,'mapping.pkl'),'wb') as f:
    pickle.dump(mapping, f)

In [ ]:
mapping['3329793486_afc16663cc']

In [ ]:
from sklearn.model_selection import train_test_split

image_ids = list(mapping.keys())

train, test = train_test_split(image_ids, test_size = 0.2, random_state = 42)

In [ ]:
import pickle

with open(os.path.join(WORKING_DIR,'mapping.pkl'),'rb') as r:
    mapping = pickle.load(r)

In [ ]:

def preprocess(mapping):
  for id,captions in mapping.items():
    for i in range(len(captions)):
      caption = captions[i]
      
      temp = []
      for word in caption.split():
        if len(word) > 1:
          temp.append(word)
      caption = "<SOS> " + ' '.join(temp) + " <EOS>"
      captions[i] = caption

preprocess(mapping)


In [ ]:
mapping['3329793486_afc16663cc']

In [ ]:
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction


actual, predicted = list(), list()

for key in tqdm(test):
    captions = mapping[key]
    #print(captions)
    
    key = key + ".jpg"
    result, _ = evaluate(key)
    
    for i in result:
      if i=="<unk>":
          result.remove(i)
    
    actual_captions = []
    
        
    for caption in captions:
        cap = list()
        words = caption.split()
        
        for word in words:
            if word in ['<SOS>','<EOS>']:
                continue
            cap.append(word)
        actual_captions.append(cap)
    
    actual.append(actual_captions)
    predicted.append(result)
    
#     print(actual)
#     print(predicted)
   

print("Bleu Score for Image Captioning(Inceptionv3)")

smoothing_function = SmoothingFunction()

print("BLEU-1: %f" % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0), smoothing_function = smoothing_function.method1))
print("BLEU-2: %f" % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0), smoothing_function = smoothing_function.method1))
    
    

In [ ]:
with open(WORKING_DIR + '/actual.pkl','wb') as a:
    pickle.dump(actual, a)
    
with open(WORKING_DIR + '/predicted.pkl','wb') as p:
    pickle.dump(predicted, p)

In [ ]:
import pickle

with open(WORKING_DIR + '/actual.pkl','rb') as f:
    actual = pickle.load(f)
    
with open(WORKING_DIR + '/predicted.pkl','rb') as r:
    predicted = pickle.load(r)

In [ ]:


from nltk.translate.bleu_score import SmoothingFunction

smoothing_function = SmoothingFunction()

print("BLEU-1: %f" % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0), smoothing_function = smoothing_function.method1))
print("BLEU-2: %f" % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0), smoothing_function = smoothing_function.method1))

In [ ]:
pip install pyciderevalcap

In [ ]:
from nltk.translate import cider

cider_scorer = cider.Cider()

# Compute CIDEr score
cider_score, _ = cider_scorer.compute_score(actual, predicted)

print("CIDEr score:", cider_score)


In [ ]:
actual.pop()
predicted.pop()

len(actual)

In [ ]:
pip install nltk


In [ ]:
from nltk.translate.bleu_score import corpus_bleu

sent_actual, sent_predicted = list(), list()

for key in tqdm(['2642350864_099c0f2152']):
    captions = mapping[key]
    #print(captions)
    
    key = key + ".jpg"
    result, _ = evaluate(key)
    
    for i in result:
      if i=="<unk>":
          result.remove(i)
            
    #result = ' '.join(result)
    
    #actual_captions = [caption.split() for caption in captions]
    
    all_caps = []
    
    for caption in captions:
        cap = list()
        words = caption.split()
        for word in words:
            if word in ['<SOS>','<EOS>']:
                continue
            cap.append(word)
        all_caps.append(cap)
            
    
    sent_actual.append(all_caps)
    sent_predicted.append(result)
    
    print(sent_actual)
    print(sent_predicted)


In [ ]:
print("BLEU-1: %f" % corpus_bleu(sent_actual, sent_predicted, weights=(1.0, 0, 0, 0)))
print("BLEU-2: %f" % corpus_bleu(sent_actual, sent_predicted, weights=(0.5, 0.5, 0, 0)))

In [ ]:
from nltk.translate.meteor_score import meteor_score

score = meteor_score(actual, predicted)

print(score)

In [ ]:
score = sentence_bleu(sent_actual, sent_predicted, weights=(1.0, 0.0, 0, 0))
print(f"BLEU score: {score}")

In [ ]:
!pip install -U nltk